In [ ]:
#----------------------------------------------------
# Code for plot error in F300M-F335M color
#
#              Jimena Rodriguez
#                    2024
#---------------------------------------------------
# Import Functions
#---------------------------------------------------
from astropy.io import ascii
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from astropy.convolution import Gaussian2DKernel, convolve
from scipy.stats import gaussian_kde

#----------------------------------------------------
#List of PHANGS-JWST Cycle 1 galaxies
#----------------------------------------------------

galaxy_names=['ngc5068','ic5332','ngc628','ngc3351','ngc3627','ngc2835','ngc4254','ngc4321','ngc4535','ngc1087','ngc4303',
          'ngc1385','ngc1566','ngc1433','ngc7496','ngc1512','ngc1300','ngc1672','ngc1365']

#----------------------------------------------------
# Read file with the F335M 5 sigma detection limit 
# These limits were computed using randomly positioned apertures, as explained in Rodriguez+25 
#----------------------------------------------------
mag_lim=ascii.read('..../ComplementaryData/Table_335_300_200_five_sigma.csv')  # your path to this files


# density percentiles that want to plot
#----------------------------------------------------
levels = [0.5, 0.7, 0.9]


figure = plt.figure(figsize=(20, 20))
figure.patch.set_facecolor('white')
x=0.01
y=0.21

for galaxy in galaxy_names:
    
    if galaxy=='ngc628':
        galaxy_p='ngc0628'
    else:
        galaxy_p=galaxy
     
    #Aperture correction values for F335M and F300M
    #----------------------------------------------------
    ac_335=-0.66
    ac_300=-0.68
    
    F335_lim=mag_lim['F335M_ab_4p'][mag_lim['galaxy']==galaxy]+ac_335
    F300_lim=mag_lim['F300M_ab_4p'][mag_lim['galaxy']==galaxy]+ac_300
    
    if galaxy=='ngc5068':
        magnitudes=np.linspace(18,30,20)
    else:
        magnitudes=np.linspace(18,28,20)
        
    #----------------------------------------------------
    # 'table' file: Contains the photometry of all sources detected in F335M.    
    #----------------------------------------------------
    table=ascii.read('../Phot_v5p4_src_335_'+galaxy+'.csv')    # your path to the photometry table 
    
        
#----------------------------------------------------    
# Density contours  
#----------------------------------------------------
    ym=np.sqrt(table['err_9010_clip_F335M']**2+table['err_9010_clip_F300M']**2)
    xm=table['F335M_ab']
    xp=xm[~np.isnan(xm) & ~np.isnan(ym) ]
    yp=ym[~np.isnan(xm) & ~np.isnan(ym) ]
    data = np.vstack([xp, yp])
    
    #estimate the probability density function of the data using a Gaussian kernel
    #----------------------------------------------------
    kde = gaussian_kde(data)

    # Evaluate the density at each point
    #----------------------------------------------------
    density = kde(data)

    # Determine the density threshold for the 50th percentile
    #----------------------------------------------------
    threshold = np.percentile(density, 50)

    # Identify points that are outside the 50th percentile contour
    #----------------------------------------------------
    outside_contour = density < threshold


    
#----------------------------------------------------    
# PLOT  
#----------------------------------------------------    
   
    # Create and plot density contours
    #----------------------------------------------------
    ax= figure.add_axes([x, y, 0.11, 0.14])
    sns.kdeplot(x=xm, y=ym, ax=ax, levels=levels, color='dimgrey', legend=False)
    
    # points outside the contours
    #---------------------------------------------------- 
    ax.scatter(xp[outside_contour], yp[outside_contour], marker='.', alpha=0.5, c='dimgrey', s=5)

    #F335M detection limit 
    #---------------------------------------------------- 
    ax.axvline(x=F335_lim, c='dodgerblue',ls='--', lw=2, label='F335M detection limit')
    
    xx=np.linspace(18,F335_lim,10)

    ax.tick_params(axis='x', direction='in', length=6)
    ax.tick_params(axis='y', direction='in', length=6) 
    ax.set_xlim(26.5,16.5)
    ax.set_ylim(-0.1,4) 
    yticks = ax.get_yticks()
    ytick_labels = ax.get_yticklabels()
    ytick_labels[-1]=''
    ax.set_xticklabels(ax.get_xticklabels(), fontsize=18)
    ax.set_yticklabels(ytick_labels, fontsize=18)
    ax.set_title(galaxy.upper(), y=0.8,loc='right',  backgroundcolor= 'silver', alpha=0.7, fontsize=20)
    
    if(x<=0.01):        
        ax.set_ylabel(r'$\rm err_{~F300M-F335M}$'+ '\n[Mag]', size=20)
    if (y>-0.2): 
            ax.set_xticklabels([])
    if ((y<-0.2) | (galaxy=='ngc7496') ):    
        ax.set_xlabel('F335M [Mag]', size=20)
    if x > 0.01:
        ax.set_yticklabels([])
        ax.set_ylabel('')
    if x > 0.01 and y > -0.3:
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_ylabel('')
    if (x>=0.33): 
        x=0.01
        y=y-0.14
    else:
        x=x+0.11
    
    if galaxy=='ngc1365':
        ax.legend(bbox_to_anchor=(1.05,0.7), loc='upper left', fontsize=18)
        
# Save plot 
#----------------------------------------------------
        
plt.savefig('.../Mag_err_color_countours_curves_points.png', transparent=False, bbox_inches='tight') # path to your output folder